In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

import momenta.utils.flux as flux
from momenta.io import GW, NuDetector, SuperNuDetector, Parameters
from momenta.io.neutrinos import  BackgroundFixed, BackgroundPoisson, EffectiveAreaAllSky
from momenta.stats.run import run_ultranest
from momenta.stats.limits import get_limits

ModuleNotFoundError: No module named 'momenta'

# Starting remarks
* For the notebook to run, you need to install `momenta` as a package (`pip install -e .` from the root directory).
* It is better to keep this code at its current location as there are some relative paths defined here, though you may modify it.

# Setup

## General parameters

In [ ]:
parameters = Parameters("input_files/config.yaml")
parameters.set_models(flux=flux.FluxFixedPowerLaw(1, 1e6, 2, eref=1))

## GW-side definitions

In [ ]:
gw = GW(
        name="GW190412", 
        path_to_fits="input_files/gw_catalogs/GW190412/GW190412_PublicationSamples.fits", 
        path_to_samples="input_files/gw_catalogs/GW190412/GW190412_subset.h5"
)
gw.set_parameters(parameters)

## Neutrino-side definitions

### Detector definitions

We use input file to define Super-Kamiokande detector but we can also use directly a dictionary to build a new detector (here with only one sample)

In [ ]:
superkamiokande = NuDetector("input_files/detector_superk.yaml")
mydetector = NuDetector({
    "name": "MyLittleDetector",
    "samples": ["MyLittleSample"],
})

### Effective area input

The detector response of any neutrino sample is especially characterized by their effective area. The effective area is the function that, when convoluted with the neutrino spectrum, gives the expected number of signal events:
\begin{equation*}
N_{\rm sig}(\Omega) = \int_{E_{\min}}^{E_{\max}} A_{\rm eff}(E, \Omega) \times \frac{dN}{dE} dE
\end{equation*}
where the effective area $A_{\rm eff}(E, \Omega)$ is expressed in ${\rm cm}^2$.

In [ ]:
# Super-Kamiokande effective areas are from https://zenodo.org/records/4724823
from superkamiokande import EffectiveAreaSK
aeffs_sk = []
for s in superkamiokande.samples:
    aeffs_sk.append(EffectiveAreaSK("input_files/effarea_superk.h5", s.name, gw.utc))
superkamiokande.set_effective_areas(aeffs_sk)

In [ ]:
# Custom effective area that is constant over the full sky and just depends on energy
class EffAreaMyDet(EffectiveAreaAllSky):
    def evaluate(self, energy, ipix, nside):
        return 1e-7 * energy**2 * np.exp(-energy/10000)
mydetector.set_effective_areas([EffAreaMyDet()])

### Observation inputs

The other main ingredients on the neutrino side are the observed and expected number of events in the different samples. The observed number of events is just an integer. The expected one may be provided in different formats that are presented below.

In [ ]:
# Super-Kamiokande background taken as a fixed value with no uncertainties
bkg_superkamiokande = [BackgroundFixed(b) for b in [0.112, 0.007, 0.016]]
# Super-Kamiokande observed number of events
nobs_superkamiokande = [0, 0, 0]

# Background from an OFF measurement where alpha_offon is the ratio between the size (time/angular) of OFF and ON regions
bkg_mydet = [BackgroundPoisson(20, alpha_offon=50)]
# Observed number of events
nobs_mydet = [0]

# Filling values to the detector objects
superkamiokande.set_observations(nobs_superkamiokande, bkg_superkamiokande)
mydetector.set_observations(nobs_mydet, bkg_mydet)

# Computing the limits per detector

### Compute limits on the flux for E^-2 spectrum

In [ ]:
parameters.flux = flux.FluxFixedPowerLaw(1, 1e6, 2, eref=1)
#
model, result = run_ultranest(superkamiokande, gw, parameters)
limit_flux_superkamiokande = get_limits(result["samples"], model)["flux0_norm"]
print(f"Limit on E^2 dN/dE for Super-Kamiokande = {limit_flux_superkamiokande} /GeV/cm²")
#
model, result = run_ultranest(mydetector, gw, parameters)
limit_flux_mydetector = get_limits(result["samples"], model)["flux0_norm"]
print(f"Limit on E^2 dN/dE for MyDetector = {limit_flux_mydetector} /GeV/cm²")

# Computing the limits combining detectors

In [ ]:
# define combined detector
combined_det = SuperNuDetector("Super-Kamiokande + MyDetector")
combined_det.add_detector(superkamiokande)
combined_det.add_detector(mydetector)

# get upper limit on flux
model, result = run_ultranest(combined_det, gw, parameters)
limit_flux_combined = get_limits(result["samples"], model)["flux0_norm"]
print(f"Limit on E^2 dN/dE for combined = {limit_flux_combined} /GeV/cm²")